In [73]:
import numpy as np
import pandas as pd
import math
import os
import re
import heapq
import jieba
from IPython.display import display

In [74]:
# 打开文件
f = open('HW4_2.txt', 'r', encoding='gbk')
# 读取文章，并删除每行结尾的换行符
docs = pd.Series(f.read().splitlines())

# 打开文件
f = open('HW4_2.txt', 'r', encoding='gbk')
# 读取文章，并删除每行结尾的换行符
sentences = pd.Series(f.read().splitlines())


In [75]:
# 添加自定义词库（可选）
for path in os.listdir('dictionary'):
    jieba.load_userdict('dictionary/' + path)

docs_ch = []
for sentence in docs:
    # 利用jieba进行分词 返回分词后的结果
    sentence = list(jieba.cut(sentence, cut_all=False))
    docs_ch.append(sentence)

for i in range(5):
    print(docs_ch[i])

['基于', '用户', '声誉', '的', '鲁棒', '协同', '推荐算法']
['基于', '覆盖', '随机', '游走', '算法', '的', '服务质量', '预测']
['基于', 'SOM', '神经网络', '的', '服务质量', '预测']
['异质信息网络', '中元', '路径', '感知', '的', '评分', '协同过滤']
['基于', '位置服务', '的', '分布式', '差分', '隐私', '推荐', '方法', '研究']


In [76]:
term_set = set()
for line in docs_ch:
    term_set.update(set(line))

idf = [0 for i in range(len(term_set))]
for i, word in enumerate(term_set):
    df = list(word in line for line in docs_ch).count(True)
    idf[i] = math.log(len(docs_ch) / df)
    
# 计算tf-idf
tf_idf = [[] for i in range(len(docs_ch))]
for i in range(len(docs_ch)):
    index, doc = i, docs_ch[i]
    
    tf = [0 if doc.count(term) == 0 else 1 + math.log(doc.count(term), 10) for term in term_set]
    
    tf_idf[i] = np.multiply(tf, idf)
    print(tf_idf[i])


[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0.         0.         0.         ... 0.         4.09434456 0.        ]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0.         0.         0.         ... 0.         4.09434456 0.        ]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0.

In [77]:
# 计算文档间余弦相似度
cosine_similaritys = [[] for i in range(len(docs_ch))]
for i in range(len(docs_ch)):
    for j in range(len(docs_ch)):
        index1, index2 = i+1, j+1
        doc1, doc2 = sentences[i], sentences[j]

        v1 = tf_idf[i]
        v2 = tf_idf[j]

        # 计算余弦相似度
        cos = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

        cosine_similaritys[i].append([cos, index1, index2, doc1, doc2])

for cosine_similarity in cosine_similaritys:
    cosine_similarity.sort(key=lambda x: x[0], reverse=True)

for cosine_similarity in cosine_similaritys[:10]:
    print('====================================================')
    for i in range(1, 4):
        print('---------------------------------------------------------------------------')
        print('第{}相似:'.format(i))
        print('文档{}和文档{}的相似度为{}'.format(cosine_similarity[i][1], cosine_similarity[i][2], round(cosine_similarity[i][0], 2)))
        print('文档{}的内容为:\t{}'.format(cosine_similarity[i][1], cosine_similarity[i][3]))
        print('文档{}的内容为:\t{}'.format(cosine_similarity[i][2], cosine_similarity[i][4]))


---------------------------------------------------------------------------
第1相似:
文档1和文档215的相似度为0.36
文档1的内容为:	基于用户声誉的鲁棒协同推荐算法
文档215的内容为:	一种基于协同矩阵分解的用户冷启动推荐算法
---------------------------------------------------------------------------
第2相似:
文档1和文档345的相似度为0.36
文档1的内容为:	基于用户声誉的鲁棒协同推荐算法
文档345的内容为:	鲁棒的单类协同排序算法
---------------------------------------------------------------------------
第3相似:
文档1和文档433的相似度为0.3
文档1的内容为:	基于用户声誉的鲁棒协同推荐算法
文档433的内容为:	基于鲁棒表示的迁移学习理论与方法研究
---------------------------------------------------------------------------
第1相似:
文档2和文档155的相似度为0.38
文档2的内容为:	基于覆盖随机游走算法的服务质量预测
文档155的内容为:	基于随机游走和多样性图排序的个性化服务推荐方法
---------------------------------------------------------------------------
第2相似:
文档2和文档3的相似度为0.37
文档2的内容为:	基于覆盖随机游走算法的服务质量预测
文档3的内容为:	基于SOM神经网络的服务质量预测
---------------------------------------------------------------------------
第3相似:
文档2和文档264的相似度为0.35
文档2的内容为:	基于覆盖随机游走算法的服务质量预测
文档264的内容为:	基于超图随机游走标签扩充的微博推荐方法
----------------------------------------------------------------